In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn import model_selection
from sklearn import linear_model
import xgboost as xgb


In [2]:
cd /Users/williamzhou/Documents/github/RussianHousing/data/best

/Users/williamzhou/Documents/github/RussianHousing/data/best


In [3]:
x_train = pd.read_csv('./x_train.csv')
y_train = pd.read_csv('./y_train.csv')
x_test  = pd.read_csv('./x_test.csv')

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(30471, 40)
(30471, 1)
(7662, 40)


In [5]:
import numpy as np
import sklearn
from sklearn import model_selection

skf = model_selection.StratifiedKFold(n_splits=3)
folds = skf.split(x_train, y_train.iloc[:,0])
_, fold1 = folds.next()
_, fold2 = folds.next()
_, fold3 = folds.next()

print('fold1 shape',len(fold1))
print('fold2 shape',len(fold2))
print('fold3 shape',len(fold3))


/Users/williamzhou/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


('fold1 shape', 17417)
('fold2 shape', 7641)
('fold3 shape', 5413)


In [6]:
fold12 = np.concatenate([fold1,fold2])
fold23 = np.concatenate([fold2,fold3])
fold13 = np.concatenate([fold1,fold3])

In [7]:
# def get_3fold_stack(model,x_train,y_train,x_test):   
# #     train fold1 and fold2
#     model.train(x_train.loc[fold12,:], y_train.loc[fold12,:])
#     new_3=model.predict(x_train.loc[fold3,:])
#     v1 = model.predict(x_test)
# #     train fold2 and fold3
#     model.train(x_train.loc[fold23,:], y_train.loc[fold23,:])
#     new_1=model.predict(x_train.loc[fold1,:])
#     v2 = model.predict(x_test)  
# #     train fold1 and fold3
#     model.train(x_train.loc[fold13,:], y_train.loc[fold13,:])
#     new_2=model.predict(x_train[fold2])
#     v3 = model.predict(x_test)

#     return np.concatenate([new1,new2,new3], axis=0), (v1+v2+v3)/3

In [10]:
def get_xgboost_3fold_stack(param,x_train,y_train,x_test):   
    print ('Train XGBoost with: {}'.format(param))
    
    dtrain_fold_combo = xgb.DMatrix(x_train.loc[fold12,:],y_train.loc[fold12,:])
    dtrain_fold = xgb.DMatrix(x_train.loc[fold3,:],y_train.loc[fold3,:])
    dtest =  xgb.DMatrix(x_test)
    model = xgb.train(dict(param, silent=0),dtrain_fold_combo)
    new_1 =  model.predict(dtrain_fold)
    v_1   =  model.predict(dtest)
    print('Finish fold12')
    
    dtrain_fold_combo = xgb.DMatrix(x_train.loc[fold23,:],y_train.loc[fold23,:])
    dtrain_fold = xgb.DMatrix(x_train.loc[fold1,:],y_train.loc[fold1,:])
    dtest =  xgb.DMatrix(x_test)
    model = xgb.train(dict(param, silent=0),dtrain_fold_combo)
    new_2 =  model.predict(dtrain_fold)
    v_2   =  model.predict(dtest)
    print('Finish fold23')
    
    dtrain_fold_combo = xgb.DMatrix(x_train.loc[fold13,:],y_train.loc[fold13,:])
    dtrain_fold = xgb.DMatrix(x_train.loc[fold2,:],y_train.loc[fold2,:])
    dtest =  xgb.DMatrix(x_test)
    model = xgb.train(dict(param, silent=0),dtrain_fold_combo)
    new_3 =  model.predict(dtrain_fold)
    v_3   =  model.predict(dtest)
    print('Finish fold13')
    
    return np.concatenate([new_1,new_2,new_3], axis=0), (v_1+v_2+v_3)/3
    print('Finish model training and prediction')

In [11]:
%%time

xgb1_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'early_stopping_rounds':20,
    'silent': 0}

xgb2_params = {
    'eta': 0.04,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0}

xgb3_params = {
    'eta': 0.02,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0}


xgboost1_train, xgboost1_test = get_xgboost_3fold_stack(xgb1_params,x_train,y_train,x_test)
xgboost2_train, xgboost2_test = get_xgboost_3fold_stack(xgb2_params,x_train,y_train,x_test)
xgboost3_train, xgboost3_test = get_xgboost_3fold_stack(xgb3_params,x_train,y_train,x_test)


Train XGBoost with: {'subsample': 0.7, 'eta': 0.05, 'colsample_bytree': 0.7, 'early_stopping_rounds': 20, 'eval_metric': 'rmse', 'objective': 'reg:linear', 'max_depth': 5, 'silent': 0}
Finish fold12
Finish fold23
Finish fold13
Train XGBoost with: {'colsample_bytree': 0.7, 'silent': 0, 'eval_metric': 'rmse', 'subsample': 0.7, 'eta': 0.04, 'objective': 'reg:linear', 'max_depth': 6}
Finish fold12
Finish fold23
Finish fold13
Train XGBoost with: {'colsample_bytree': 0.7, 'silent': 0, 'eval_metric': 'rmse', 'subsample': 1.0, 'eta': 0.02, 'objective': 'reg:linear', 'max_depth': 5}
Finish fold12
Finish fold23
Finish fold13
CPU times: user 5.56 s, sys: 188 ms, total: 5.75 s
Wall time: 5.75 s


In [40]:
X_train_stack = np.concatenate([x_train, 
                                pd.DataFrame(xgboost1_train),
                                pd.DataFrame(xgboost2_train),
                                pd.DataFrame(xgboost3_train)], axis=1)
xgtrain_stack = xgb.DMatrix(X_train_stack,y_train)


X_test_stack = np.concatenate([x_test, 
                                pd.DataFrame(xgboost1_test),
                                pd.DataFrame(xgboost2_test),
                                pd.DataFrame(xgboost3_test)], axis=1)

xgtest_stack = xgb.DMatrix(X_test_stack)

In [37]:
final_xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0}

cv_output = xgb.cv(final_xgb_params, xgtrain_stack, 
                       num_boost_round=1000, 
                       early_stopping_rounds=20,
                       verbose_eval=50, show_stdv=False)
test_rmse = cv_output.loc[len(cv_output)-1,'test-rmse-mean']
print(test_rmse)  
num_boost_rounds = len(cv_output)
model = xgb.train(dict(final_xgb_params, silent=0), xgtrain_stack, num_boost_round=num_boost_rounds)


[0]	train-rmse:134051	test-rmse:134062
[50]	train-rmse:40935.2	test-rmse:42592
[100]	train-rmse:37504.3	test-rmse:40429
[150]	train-rmse:36038.5	test-rmse:39962.8
[200]	train-rmse:34903.2	test-rmse:39713.3
[250]	train-rmse:33872.9	test-rmse:39563.5
[300]	train-rmse:32925.9	test-rmse:39473.1
[350]	train-rmse:32089.1	test-rmse:39415.8
[400]	train-rmse:31332.3	test-rmse:39382.8
[450]	train-rmse:30585.4	test-rmse:39372.5
39365.377604


In [41]:
y_predict = model.predict(xgtest_stack)
y_predic_all_sq = (y_predict)*x_test['full_sq']
df_sub = pd.DataFrame({'id': range(30474,30474+7662), 'price_doc': y_predic_all_sq})
df_sub.to_csv('./sub.csv', index=False)
df_sub.head()

,id,price_doc
0,30474,5.498277e+06
1,30475,7.911552e+06
2,30476,5.083375e+06
3,30477,5.639118e+06
4,30478,4.642392e+06
